# Graph Visualization

<a href="https://githubtocolab.com/gleanerio/archetype/blob/master/networks/commons/notebooks/networkViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.png" alt="Open in Colab"/></a>

## requirements.txt


In [ ]:
!pip install -q minio
!pip install -q kglab
!pip install -q sparqlwrapper
!pip install -q pygraphml
!pip install -q ipysigma

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
import kglab
from minio import Minio
from rdflib import Graph, plugin
import plotly.express as px
import pandas as pd
from urllib.request import urlopen
import os,json

from pygraphml import GraphMLParser
from pygraphml import Graph as GraphML 
import networkx as nx
from ipysigma import Sigma
from SPARQLWrapper import SPARQLWrapper, JSON
from hashlib import sha256

In [2]:
def hash_text(text):
    return sha256(text.encode()).hexdigest()[:8]

In [3]:
## Remote SPARQL server
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)



In [45]:
rp1 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT ?source ?type ?target ?sType ?tType ?sName ?tName
WHERE {
  ?source a ?sType .
  OPTIONAL { ?source schema:name ?sName . }
  
  ?target a ?tType .
  OPTIONAL { ?target schema:name ?tName .}
  
   ?source ?type ?target .

}
"""

ep = "http://0.0.0.0:7878/query"

df = get_sparql_dataframe(ep, rp1)


In [46]:
df.to_parquet('inspiretypes.parquet')
# df = pd.read_parquet('path/to/your/file.parquet')

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  115 non-null    object
 1   type    115 non-null    object
 2   target  115 non-null    object
 3   sType   115 non-null    object
 4   tType   115 non-null    object
 5   sName   73 non-null     object
 6   tName   58 non-null     object
dtypes: object(7)
memory usage: 6.4+ KB


In [48]:
df.head(10)

,source,type,target,sType,tType,sName,tName
0,c5750d9d43bd89e4be35d52609e63d83,https://schema.org/softwareHelp,fd2c3503bda483563df599f0ba1dd7ed,https://schema.org/SoftwareApplication,https://schema.org/CreativeWork,HADES (Health Analytics Data-To-Evidence Suite),The Book of OHDSI: Chapter 13
1,c5750d9d43bd89e4be35d52609e63d83,https://aphrc.org/analysisModelDetails,9715e6df5aa1ab430704026bc2c72a09,https://schema.org/SoftwareApplication,https://schema.org/ItemList,HADES (Health Analytics Data-To-Evidence Suite),AnalysisModelDetails
2,https://example.org/id/concept/317009,https://schema.org/increasesRiskOf,73cac76f78250f726d218144ff703615,https://schema.org/MedicalRiskFactor,https://schema.org/MedicalCondition,Asthma,None
3,https://example.org/id/concept/317009,https://schema.org/code,ffb86f2a3f98edbeb4ecb55a8c8ee487,https://schema.org/MedicalRiskFactor,https://schema.org/MedicalCode,Asthma,None
4,https://example.org/id/concept/201820,https://schema.org/increasesRiskOf,117d68aa50e4cffc8748552b8981a67a,https://schema.org/MedicalRiskFactor,https://schema.org/MedicalCondition,Diabetes mellitus,None
5,https://example.org/id/concept/201820,https://schema.org/code,659dba644bd3d45cb34d611363766783,https://schema.org/MedicalRiskFactor,https://schema.org/MedicalCode,Diabetes mellitus,None
6,12457db04750af96b849f334fc8f38fa,https://schema.org/item,803c0b091deb8901e2331b56608ad5e3,https://schema.org/ListItem,https://schema.org/MedicalIndication,None,None
7,188edf84febe2788731946f7448b8895,https://schema.org/item,b045b16e02a8c670544d30041e34db68,https://schema.org/ListItem,https://schema.org/ItemList,None,None
8,282653eef48a4ba55ba0506081e60f9b,https://schema.org/item,4828b0ed8738c740f2f44ccb9b3ac4ef,https://schema.org/ListItem,https://schema.org/SoftwareApplication,HADES,None
9,2e2b80b723f0f53ccc6a528c51166c10,https://schema.org/item,1aa0b03f7c735272fddeca57b8c3de13,https://schema.org/ListItem,https://schema.org/SoftwareApplication,ACHILLES,None


In [49]:
#  yeah, I get it...   don't iterate rows...  PR's welcome for this, being bad is too easy!  
# for index, row in nodes.iterrows():
#    g.add_node(row['Id'])
g = GraphML()
g.directed = False

for index, row in df.iterrows():
    n1 = g.add_node(row['source'])
    n1['type'] = row['sType']
    n1['name'] = row['sName']
    
    n2 = g.add_node(row['target'])
    n2['type'] = row['tType']
    n2['name'] = row['tName']

    e1 = g.add_edge(n1, n2)
    
fname = "./inspire.graphml"
parser = GraphMLParser()
parser.write(g, fname)

In [50]:
# g = nx.les_miserables_graph()
g = nx.read_graphml("./inspire.graphml")

In [51]:
Sigma(
    g,
    node_size=g.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='louvain',
    start_layout=5,
    edge_size=lambda u, v: g.degree(u) + g.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=g.degree,
    label_density=0
)



Sigma(nx.Graph with 117 nodes and 115 edges)